# Módulo 2: Técnicas Avançadas de Recuperação de Informação
Script 2.1: Implementação de Recuperação Neural com Representações Densas
Este script demonstra como utilizar embeddings densos para melhorar a recuperação de documentos por meio de recuperação neural.

In [22]:
# Instalação das bibliotecas necessárias (execute apenas se ainda não estiverem instaladas)
#!pip install -q sentence-transformers transformers
#!pip install --upgrade jupyter ipywidgets

In [23]:
# Importação das bibliotecas necessárias
#!pip install jupyterlab
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager
#!jupyter nbextension enable --py widgetsnbextension
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import torch


In [24]:
# Carregamento de um conjunto de documentos de exemplo
documents = [
    "O aprendizado de máquina é uma subárea da inteligência artificial focada em algoritmos.",
    "Redes neurais são modelos computacionais inspirados no cérebro humano.",
    "A recuperação de informação trata de obter informações relevantes a partir de grandes conjuntos de dados.",
    "Modelos de linguagem pré-treinados podem gerar texto semelhante ao humano.",
    "A engenharia de prompt é crucial para orientar modelos de linguagem a produzirem respostas desejadas."
]


In [25]:
torch.cuda.empty_cache()

In [26]:
# Limpar a memória CUDA primeiro
torch.cuda.empty_cache()

# Forçar o uso da CPU
model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')

# Cálculo dos embeddings dos documentos usando CPU
doc_embeddings = model.encode(documents, convert_to_tensor=True)

In [27]:
# Definição da consulta
consulta = "Como os modelos de linguagem geram texto semelhante ao humano?"

# Cálculo do embedding da consulta
consulta_embedding = model.encode(consulta, convert_to_tensor=True)

# Cálculo das similaridades
cosine_scores = util.cos_sim(consulta_embedding, doc_embeddings)[0]

# Obtenção dos índices dos documentos mais similares
top_k = 3
top_results = torch.topk(cosine_scores, k=top_k)

# Exibição dos documentos recuperados
print("Documentos Recuperados:")
for score, idx in zip(top_results.values, top_results.indices):
    print(f"(Score: {score:.4f}) - {documents[idx]}")


Documentos Recuperados:
(Score: 0.8946) - Modelos de linguagem pré-treinados podem gerar texto semelhante ao humano.
(Score: 0.6423) - A engenharia de prompt é crucial para orientar modelos de linguagem a produzirem respostas desejadas.
(Score: 0.5441) - Redes neurais são modelos computacionais inspirados no cérebro humano.


In [28]:
# Uso de um modelo de linguagem para gerar a resposta
generator = pipeline('text-generation', model='distilgpt2')

# Concatenando os documentos recuperados
contexto = " ".join([documents[idx] for idx in top_results.indices])

# Construção do prompt para o modelo de linguagem
prompt = f"Contexto: {contexto}\nPergunta: {consulta}\nResposta:"

# Geração da resposta
resposta = generator(prompt, max_new_tokens=150, num_return_sequences=1)

# Exibição da resposta
print("\nResposta Gerada:")
print(resposta[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Resposta Gerada:
Contexto: Modelos de linguagem pré-treinados podem gerar texto semelhante ao humano. A engenharia de prompt é crucial para orientar modelos de linguagem a produzirem respostas desejadas. Redes neurais são modelos computacionais inspirados no cérebro humano.
Pergunta: Como os modelos de linguagem geram texto semelhante ao humano?
Resposta: Me gese ai por oque en el mundo oque la nacional.
Cui de gesquierdo: Como os modelos de linguagem açao, es ação. Añalo por sono, da ejeção tenga ser férín.
Udo nesque de sua, para ação en udo ném que udo nesque desejadas.
Egoção: Oquia, y todos estamos éjos no ajestos egoção.
Udo céres que ação ação de mez peremção:


Análise:

Melhorias Observadas:
A recuperação neural captura melhor o contexto semântico, recuperando documentos mais relevantes.
A resposta gerada tende a ser mais precisa devido à melhor qualidade dos documentos recuperados.

## Script 2.2: Implementação de Pesquisa Híbrida (Combinação de Métodos Esparsos e Densos)
Este script demonstra como combinar métodos esparsos (BM25) e densos (embeddings) para melhorar a recuperação de documentos.


In [29]:
# Importação das bibliotecas necessárias
from sklearn.preprocessing import MinMaxScaler
import numpy as np


In [30]:
# Importações adicionais necessárias
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import nltk

# Download necessário para o NLTK (execute apenas uma vez)
nltk.download('punkt')

# Tokenização dos documentos
tokenized_docs = [word_tokenize(doc.lower()) for doc in documents]

# Criação do objeto BM25
bm25 = BM25Okapi(tokenized_docs)

# Tokenização da consulta
tokenized_query = word_tokenize(consulta.lower())

# Agora podemos calcular os scores BM25
bm25_scores = bm25.get_scores(tokenized_query)

# Normalização dos scores BM25
scaler = MinMaxScaler()
bm25_scores_norm = scaler.fit_transform(bm25_scores.reshape(-1, 1)).flatten()

[nltk_data] Downloading package punkt to /home/anderson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [31]:
# Cálculo das similaridades densas (Embeddings)
cosine_scores = util.cos_sim(consulta_embedding, doc_embeddings)[0].cpu().numpy()

# Normalização das similaridades densas
cosine_scores_norm = scaler.fit_transform(cosine_scores.reshape(-1, 1)).flatten()


In [32]:
# Combinação das similaridades
alpha = 0.5  # Peso para a similaridade esparsa
beta = 0.5   # Peso para a similaridade densa

combined_scores = alpha * bm25_scores_norm + beta * cosine_scores_norm

# Obtenção dos índices dos documentos mais similares
top_k = 3
combined_top_indices = combined_scores.argsort()[-top_k:][::-1]

# Exibição dos documentos recuperados
print("Documentos Recuperados (Pesquisa Híbrida):")
for idx in combined_top_indices:
    print(f"(Score: {combined_scores[idx]:.4f}) - {documents[idx]}")


Documentos Recuperados (Pesquisa Híbrida):
(Score: 1.0000) - Modelos de linguagem pré-treinados podem gerar texto semelhante ao humano.
(Score: 0.3396) - A engenharia de prompt é crucial para orientar modelos de linguagem a produzirem respostas desejadas.
(Score: 0.2319) - Redes neurais são modelos computacionais inspirados no cérebro humano.


In [33]:
# Uso do modelo de linguagem para gerar a resposta
generator = pipeline('text-generation', model='distilgpt2')

# Concatenando os documentos recuperados
contexto = " ".join([documents[idx] for idx in combined_top_indices])

# Construção do prompt para o modelo de linguagem
prompt = f"Contexto: {contexto}\nPergunta: {consulta}\nResposta:"

# Geração da resposta
resposta = generator(prompt, max_new_tokens=150, num_return_sequences=1)

# Exibição da resposta
print("\nResposta Gerada:")
print(resposta[0]['generated_text'])


/home/anderson/miniconda3/envs/RAG/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Resposta Gerada:
Contexto: Modelos de linguagem pré-treinados podem gerar texto semelhante ao humano. A engenharia de prompt é crucial para orientar modelos de linguagem a produzirem respostas desejadas. Redes neurais são modelos computacionais inspirados no cérebro humano.
Pergunta: Como os modelos de linguagem geram texto semelhante ao humano?
Resposta: São en una por ôn o ôn.


# Análise Detalhada da Implementação de Busca Híbrida (BM25 + Embeddings Densos)

## 1. Visão Geral
O código implementa uma estratégia de busca híbrida que combina dois métodos de recuperação de informação:
- **Método Esparso**: BM25 (Best Matching 25)
- **Método Denso**: Embeddings usando Sentence Transformers

## 2. Implementação do BM25 (Método Esparso)
```python
# Preparação dos documentos
tokenized_docs = [word_tokenize(doc.lower()) for doc in documents]
bm25 = BM25Okapi(tokenized_docs)

# Processamento da consulta
tokenized_query = word_tokenize(consulta.lower())
bm25_scores = bm25.get_scores(tokenized_query)

# Normalização dos scores
bm25_scores_norm = scaler.fit_transform(bm25_scores.reshape(-1, 1)).flatten()
```
- O BM25 trabalha com tokens (palavras individuais)
- Cada documento é tokenizado e convertido para minúsculas
- A consulta passa pelo mesmo processo de tokenização
- Os scores são normalizados para escala [0,1] usando MinMaxScaler

## 3. Embeddings Densos (Método Denso)
```python
# Cálculo das similaridades densas
cosine_scores = util.cos_sim(consulta_embedding, doc_embeddings)[0].cpu().numpy()
cosine_scores_norm = scaler.fit_transform(cosine_scores.reshape(-1, 1)).flatten()
```
- Utiliza o modelo `all-MiniLM-L6-v2` para gerar embeddings
- Calcula similaridade por cosseno entre consulta e documentos
- Também normaliza os scores para escala [0,1]

## 4. Combinação dos Métodos
```python
alpha = 0.5  # Peso para similaridade esparsa (BM25)
beta = 0.5   # Peso para similaridade densa (Embeddings)
combined_scores = alpha * bm25_scores_norm + beta * cosine_scores_norm
```
- Usa média ponderada dos scores normalizados
- Pesos iguais (0.5) para ambos os métodos
- Permite ajuste dos pesos conforme necessidade

## 5. Resultados e Geração de Resposta
Os resultados mostram que a busca híbrida recuperou documentos relevantes:
1. "Modelos de linguagem pré-treinados podem gerar texto semelhante ao humano." (Score: 1.0000)
2. "A engenharia de prompt é crucial para orientar modelos de linguagem..." (Score: 0.3396)
3. "Redes neurais são modelos computacionais..." (Score: 0.2319)

## 6. Observações sobre a Geração de Resposta
- A resposta gerada pelo modelo distilgpt2 não foi satisfatória
- O modelo está gerando texto sem sentido, possivelmente porque:
  1. Está sendo usado sem fine-tuning para português
  2. O prompt pode precisar de melhor engenharia
  3. O modelo distilgpt2 pode não ser a melhor escolha para português

## 7. Vantagens da Abordagem Híbrida
1. **BM25**: Excelente em correspondência exata de palavras-chave
2. **Embeddings**: Captura relações semânticas e contextuais
3. **Combinação**: Aproveita o melhor dos dois mundos, melhorando a qualidade da recuperação

## 8. Possíveis Melhorias
1. Ajustar os pesos alpha e beta
2. Usar um modelo de linguagem em português
3. Melhorar a engenharia de prompt
4. Implementar validação cruzada para otimizar os pesos